## AutoGen Studio Agent Workflow API Example

This notebook focuses on demonstrating capabilities of the autogen studio workflow python api.  

- Declarative Specification of an Agent Team
- Loading the specification and running the resulting agent

 

In [ ]:
from autogenstudio.teammanager import TeamManager

wm = TeamManager()
result = await wm.run(task="What is the weather in New York?", team_config="team.json")
print(result)

In [ ]:
result_stream =  wm.run_stream(task="What is the weather in New York?", team_config="team.json")
async for response in result_stream:
    print(response)

## Load Directly Using the AgentChat API 

In [ ]:
import json 
from autogen_agentchat.teams import BaseGroupChat
team_config = json.load(open("team.json"))  
team = BaseGroupChat.load_component(team_config)
print(team._participants)

## AutoGen Studio Database API

Api for creating objects and serializing to a database.

In [ ]:
from autogenstudio.database import DatabaseManager
import os
# delete database
# if os.path.exists("test.db"):
#     os.remove("test.db")

os.makedirs("test", exist_ok=True)
# create a database
dbmanager = DatabaseManager(engine_uri="sqlite:///test.db", base_dir="test")
dbmanager.initialize_database()

## Sample AgentChat Example (Python)

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

planner_agent = AssistantAgent(
    "planner_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request. Respond with a single sentence",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided. Respond with a single sentence",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.Respond with a single sentence",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed tfinal travel plan. You must ensure th b at the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.Respond with a single sentence",
)

termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(10)
group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)




In [ ]:
result = group_chat.run_stream(task="Plan a 3 day trip to Nepal.")
async for response in result:
    print(response)

In [ ]:
import json

# convert to config 
config = group_chat.dump_component().model_dump()
# save as json 

with open("travel_team.json", "w") as f:
    json.dump(config, f, indent=4)

# load from json
with open("travel_team.json", "r") as f:
    config = json.load(f)

group_chat = RoundRobinGroupChat.load_component(config) 
result = group_chat.run_stream(task="Plan a 3 day trip to Nepal.") 
async for response in result:
    print(response)

In [ ]:
from autogenstudio.teammanager import TeamManager

wm = TeamManager() 
# Run a task using the team configuration
result_stream =  wm.run_stream(task="What is the weather in New York?", team_config="mcp_team.json")
async for response in result_stream:
    print(response)

In [ ]:
import asyncio
from datetime import timedelta
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import StreamableHttpMcpToolAdapter, StreamableHttpServerParams
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.tools.mcp import McpWorkbench
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
 
    # Create server params for the remote MCP service
server_params = StreamableHttpServerParams(
    url="http://localhost:8005/mcp",
    headers={"Authorization": "Bearer your-api-key", "Content-Type": "application/json"},
    timeout=30,
    sse_read_timeout= 60 * 5,
    terminate_on_close=True,
)

workbench = McpWorkbench(server_params=server_params)
 

# Create an agent that can use the translation tool
model_client = OpenAIChatCompletionClient(model="gpt-4")
agent = AssistantAgent(
    name="translator",
    model_client=model_client, 
    workbench=workbench,
    system_message="You are a helpful translation assistant.",
)
team = RoundRobinGroupChat(
    [agent],
    termination_condition= TextMentionTermination("TERMINATE") | MaxMessageTermination(10),
)

await Console(agent.run_stream(task="What is the weather in New York?"))

---------- TextMessage (user) ----------
What is the weather in New York?


In [ ]:
workbench.dump_component()

In [ ]:
import json 
# # save as json 

with open("mcp_team.json", "w") as f:
    json.dump(team.dump_component().model_dump(), f, indent=4) 

import json